# Word Embeddings: Encoding Lexical Semantics





Two examples in this notebook are considered:

1.   N-Gram Language Modeling
2.   Continuous Bag-of-Words

Based on a notebook of Robert Guthrie.





Word embeddings are dense vectors of real numbers, one per word in your
vocabulary. In NLP, it is almost always the case that your features are
words! But how should you represent a word in a computer? You could
store its ascii character representation, but that only tells you what
the word *is*, it doesn't say much about what it *means* (you might be
able to derive its part of speech from its affixes, or properties from
its capitalization, but not much). Even more, in what sense could you
combine these representations? We often want dense outputs from our
neural networks, where the inputs are $|V|$ dimensional, where
$V$ is our vocabulary, but often the outputs are only a few
dimensional (if we are only predicting a handful of labels, for
instance). How do we get from a massive dimensional space to a smaller
dimensional space?

How about instead of ascii representations, we use a one-hot encoding?
That is, we represent the word $w$ by

\begin{align}\overbrace{\left[ 0, 0, \dots, 1, \dots, 0, 0 \right]}^\text{|V| elements}\end{align}

where the 1 is in a location unique to $w$. Any other word will
have a 1 in some other location, and a 0 everywhere else.

There is an enormous drawback to this representation, besides just how
huge it is. It basically treats all words as independent entities with
no relation to each other. What we really want is some notion of
*similarity* between words. Why? Let's see an example.

Suppose we are building a language model. Suppose we have seen the
sentences

* The mathematician ran to the store.
* The physicist ran to the store.
* The mathematician solved the open problem.

in our training data. Now suppose we get a new sentence never before
seen in our training data:

* The physicist solved the open problem.

Our language model might do OK on this sentence, but wouldn't it be much
better if we could use the following two facts:

* We have seen  mathematician and physicist in the same role in a sentence. Somehow they
  have a semantic relation.
* We have seen mathematician in the same role  in this new unseen sentence
  as we are now seeing physicist.

and then infer that physicist is actually a good fit in the new unseen
sentence? This is what we mean by a notion of similarity: we mean
*semantic similarity*, not simply having similar orthographic
representations. It is a technique to combat the sparsity of linguistic
data, by connecting the dots between what we have seen and what we
haven't. This example of course relies on a fundamental linguistic
assumption: that words appearing in similar contexts are related to each
other semantically. This is called the [distributional
hypothesis](https://en.wikipedia.org/wiki/Distributional_semantics).

## Getting Dense Word Embeddings

How can we solve this problem? That is, how could we actually encode
semantic similarity in words? Maybe we think up some semantic
attributes. For example, we see that both mathematicians and physicists
can run, so maybe we give these words a high score for the "is able to
run" semantic attribute. Think of some other attributes, and imagine
what you might score some common words on those attributes.

If each attribute is a dimension, then we might give each word a vector,
like this:

$$
q_\text{mathematician} = \left[ \overbrace{2.3}^\text{can run},
   \overbrace{9.4}^\text{likes coffee}, \overbrace{-5.5}^\text{majored in Physics}, \dots \right]
$$

$$
q_\text{physicist} = \left[ \overbrace{2.5}^\text{can run},
   \overbrace{9.1}^\text{likes coffee}, \overbrace{6.4}^\text{majored in Physics}, \dots \right]
$$

Then we can get a measure of similarity between these words by doing:
$$
\text{Similarity}(\text{physicist}, \text{mathematician}) = q_\text{physicist} \cdot q_\text{mathematician}
$$

Although it is more common to normalize by the lengths:

$$
\text{Similarity}(\text{physicist}, \text{mathematician}) = \frac{q_\text{physicist} \cdot q_\text{mathematician}}
   {\| q_\text{physicist} \| \| q_\text{mathematician} \|} = \cos (\phi)
$$
where $\phi$ is the angle between the two vectors. That way,
extremely similar words (words whose embeddings point in the same
direction) will have similarity 1. Extremely dissimilar words should
have similarity -1.


You can think of the sparse one-hot vectors from the beginning of this
section as a special case of these new vectors we have defined, where
each word basically has similarity 0, and we gave each word some unique
semantic attribute. These new vectors are *dense*, which is to say their
entries are (typically) non-zero.

But these new vectors are a big pain: you could think of thousands of
different semantic attributes that might be relevant to determining
similarity, and how on earth would you set the values of the different
attributes? Central to the idea of deep learning is that the neural
network learns representations of the features, rather than requiring
the programmer to design them herself. So why not just let the word
embeddings be parameters in our model, and then be updated during
training? This is exactly what we will do. We will have some *latent
semantic attributes* that the network can, in principle, learn. Note
that the word embeddings will probably not be interpretable. That is,
although with our hand-crafted vectors above we can see that
mathematicians and physicists are similar in that they both like coffee,
if we allow a neural network to learn the embeddings and see that both
mathematicians and physicists have a large value in the second
dimension, it is not clear what that means. They are similar in some
latent semantic dimension, but this probably has no interpretation to
us.


In summary, **word embeddings are a representation of the *semantics* of
a word, efficiently encoding semantic information that might be relevant
to the task at hand**. You can embed other things too: part of speech
tags, parse trees, anything! The idea of feature embeddings is central
to the field.

## Word Embeddings in Pytorch

Similar to how we defined a unique index for each word when
making one-hot vectors, we also need to define an index for each word
when using embeddings. These will be keys into a lookup table. That is,
embeddings are stored as a $|V| \times D$ matrix, where $D$
is the dimensionality of the embeddings, such that the word assigned
index $i$ has its embedding stored in the $i$'th row of the
matrix. In all of my code, the mapping from words to indices is a
dictionary named word\_to\_ix.

The module that allows you to use embeddings is torch.nn.Embedding,
which takes two arguments: the vocabulary size, and the dimensionality
of the embeddings.

To index into this table, you must use torch.LongTensor (since the
indices are integers, not floats).

In [0]:
%matplotlib inline

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [3]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward>)


In [4]:
lookup_tensor = torch.tensor([word_to_ix["world"]], dtype=torch.long)
world_embed = embeds(lookup_tensor)
print(world_embed)

tensor([[-0.1661, -1.5228,  0.3817, -1.0276, -0.5631]],
       grad_fn=<EmbeddingBackward>)


In [5]:
[word_to_ix["hello"]]

[0]

In [6]:
lookup_tensor = torch.tensor([word_to_ix["world"]], dtype=torch.long)
world_embed = embeds(lookup_tensor)
print(world_embed)

tensor([[-0.1661, -1.5228,  0.3817, -1.0276, -0.5631]],
       grad_fn=<EmbeddingBackward>)


## First Example: N-Gram Language Modeling


Recall that in an n-gram language model, given a sequence of words
$w$, we want to compute
$$
P(w_i | w_{i-1}, w_{i-2}, \dots, w_{i-n+1} )
$$
where $w_i$ is the ith word of the sequence.

In this example, we will compute the loss function on some training
examples and update the parameters with backpropagation.





In [7]:

# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
# print the first 5, just so you can see what they look like
print("3-gram subset:", trigrams[:5])

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}

print("dictionary: ", word_to_ix)

3-gram subset: [(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege'), (['shall', 'besiege'], 'thy'), (['besiege', 'thy'], 'brow,')]
dictionary:  {'days;': 0, 'thriftless': 1, 'shame,': 2, 'mine': 3, 'gazed': 4, 'say,': 5, 'an': 6, 'his': 7, 'besiege': 8, 'more': 9, 'praise': 10, 'thine!': 11, 'Thy': 12, 'lies,': 13, 'of': 14, 'child': 15, 'answer': 16, 'within': 17, 'Proving': 18, 'And': 19, 'it': 20, 'asked,': 21, 'thy': 22, 'deep': 23, 'eyes,': 24, 'brow,': 25, 'small': 26, 'art': 27, 'winters': 28, 'held:': 29, "deserv'd": 30, 'to': 31, 'Will': 32, 'worth': 33, 'warm': 34, 'own': 35, 'Shall': 36, 'livery': 37, 'Where': 38, 'by': 39, 'If': 40, 'fair': 41, 'When': 42, 'trenches': 43, 'shall': 44, 'dig': 45, "youth's": 46, 'see': 47, 'in': 48, 'Were': 49, 'count,': 50, 'were': 51, "totter'd": 52, 'new': 53, 'being': 54, 'so': 55, 'now,': 56, 'old': 57, 'blood': 58, 'couldst': 59, 'How': 60, 'succession': 61, 'sum': 62, 'praise.': 63, 'thine

In [8]:
vocab

{"'This",
 'And',
 'How',
 'If',
 'Proving',
 'Shall',
 'Then',
 'This',
 'Thy',
 'To',
 'Were',
 'When',
 'Where',
 'Will',
 'a',
 'all',
 'all-eating',
 'an',
 'and',
 'answer',
 'art',
 'asked,',
 'be',
 'beauty',
 "beauty's",
 'being',
 'besiege',
 'blood',
 'brow,',
 'by',
 'child',
 'cold.',
 'couldst',
 'count,',
 'days;',
 'deep',
 "deserv'd",
 'dig',
 "excuse,'",
 'eyes,',
 'fair',
 "feel'st",
 'field,',
 'forty',
 'gazed',
 'held:',
 'his',
 'in',
 'it',
 'lies,',
 'livery',
 'lusty',
 'made',
 'make',
 'mine',
 'more',
 'much',
 'my',
 'new',
 'now,',
 'of',
 'old',
 'old,',
 'on',
 'own',
 'praise',
 'praise.',
 'proud',
 'say,',
 'see',
 'shall',
 'shame,',
 'small',
 'so',
 'succession',
 'sum',
 'sunken',
 'the',
 'thine',
 'thine!',
 'thou',
 'thriftless',
 'thy',
 'to',
 "totter'd",
 'treasure',
 'trenches',
 'use,',
 'warm',
 'weed',
 'were',
 'when',
 'where',
 'winters',
 'within',
 'worth',
 "youth's"}

In [9]:
len(vocab)

97

In [23]:
trigrams

[(['When', 'forty'], 'winters'),
 (['forty', 'winters'], 'shall'),
 (['winters', 'shall'], 'besiege'),
 (['shall', 'besiege'], 'thy'),
 (['besiege', 'thy'], 'brow,'),
 (['thy', 'brow,'], 'And'),
 (['brow,', 'And'], 'dig'),
 (['And', 'dig'], 'deep'),
 (['dig', 'deep'], 'trenches'),
 (['deep', 'trenches'], 'in'),
 (['trenches', 'in'], 'thy'),
 (['in', 'thy'], "beauty's"),
 (['thy', "beauty's"], 'field,'),
 (["beauty's", 'field,'], 'Thy'),
 (['field,', 'Thy'], "youth's"),
 (['Thy', "youth's"], 'proud'),
 (["youth's", 'proud'], 'livery'),
 (['proud', 'livery'], 'so'),
 (['livery', 'so'], 'gazed'),
 (['so', 'gazed'], 'on'),
 (['gazed', 'on'], 'now,'),
 (['on', 'now,'], 'Will'),
 (['now,', 'Will'], 'be'),
 (['Will', 'be'], 'a'),
 (['be', 'a'], "totter'd"),
 (['a', "totter'd"], 'weed'),
 (["totter'd", 'weed'], 'of'),
 (['weed', 'of'], 'small'),
 (['of', 'small'], 'worth'),
 (['small', 'worth'], 'held:'),
 (['worth', 'held:'], 'Then'),
 (['held:', 'Then'], 'being'),
 (['Then', 'being'], 'asked

### Question: what is the role of "nn.Embedding"?

**Answer:** 

Generate a simple lookup table that stores embeddings of a fixed dictionary and size, that is going to be used for the neural network.

In [11]:
CONTEXT_SIZE = 2 # indicates how many words will be used in the context
EMBEDDING_DIM = 10

class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in trigrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

[518.8735959529877, 516.5372025966644, 514.2159497737885, 511.91007828712463, 509.61842346191406, 507.3396894931793, 505.0738744735718, 502.82053303718567, 500.57798433303833, 498.34599709510803]


In [12]:
print(context)
print(context_idxs)

["feel'st", 'it']
tensor([91, 20])


In [14]:
context = ['when', 'thou']
context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
context_idxs
#log_probs = model(context_idxs)

tensor([74, 71])

## Second Example: Continuous Bag-of-Words


The Continuous Bag-of-Words model (CBOW) is frequently used in NLP deep
learning. It is a model that tries to predict words given the context of
a few words before and a few words after the target word. This is
distinct from language modeling, since CBOW is not sequential and does
not have to be probabilistic. Typcially, CBOW is used to quickly train
word embeddings, and these embeddings are used to initialize the
embeddings of some more complicated model. Usually, this is referred to
as *pretraining embeddings*. It almost always helps performance a couple
of percent.

The CBOW model is as follows. Given a target word $w_i$ and an
$N$ context window on each side, $w_{i-1}, \dots, w_{i-N}$
and $w_{i+1}, \dots, w_{i+N}$, referring to all context words
collectively as $C$, CBOW tries to minimize

\begin{align}-\log p(w_i | C) = -\log \text{Softmax}(A(\sum_{w \in C} q_w) + b)\end{align}

where $q_w$ is the embedding for word $w$.



### Question: what is the role of "word_to_ix" and "ix_to_word"?

**Answer:**
word_to_ix -> access an index for the vocabulary given an word
ix_to_word -> access the word in the vocabulary given an index

In [15]:
import torch
import torch.nn as nn
import numpy as np

def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

def get_index_of_max(input):
    index = 0
    for i in range(1, len(input)):
        if input[i] > input[index]:
            index = i 
    return index

def get_max_prob_result(input, ix_to_word):
    return ix_to_word[get_index_of_max(input)]


CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
EMDEDDING_DIM = 100

word_to_ix = {}
ix_to_word = {}

raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()


# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)
print(vocab_size)

for i, word in enumerate(vocab):
    word_to_ix[word] = i
    ix_to_word[i] = word



49


In [16]:
try:
  print(word_to_ix['spells'])
except: 
  print('Word spells not found. Check if there is any punctuation marks!')

try:
  print("The word at position 5 is:", ix_to_word[5])
except:
  print('Index 5 is out of range!')

Word spells not found. Check if there is any punctuation marks!
The word at position 5 is: to


In [17]:
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
    
    
print(context)    

['the', 'computer', 'our', 'spells.']


Class CBOW

### Question: what is the difference between the CBOW architecture coded in the following CBOW class and the CBOW architecture described in the lecture?

**Answer:**

Here we have two layers.

In [0]:
class CBOW(torch.nn.Module):

    def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()

        #out: 1 x emdedding_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)

        self.linear1 = nn.Linear(embedding_dim, 128)

        self.activation_function1 = nn.ReLU()
        
        #out: 1 x vocab_size
        self.linear2 = nn.Linear(128, vocab_size)

        self.activation_function2 = nn.LogSoftmax(dim = -1)
        

    def forward(self, inputs):
        embeds = sum(self.embeddings(inputs)).view(1,-1)
        out = self.linear1(embeds)
        out = self.activation_function1(out)
        out = self.linear2(out)
        out = self.activation_function2(out)
        return out

    def get_word_embedding(self, word):
        word = torch.LongTensor([word_to_ix[word]])
        return self.embeddings(word).view(1,-1)





Train the CBOW model

In [0]:
model = CBOW(vocab_size, EMDEDDING_DIM)

loss_function = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)


for epoch in range(50):
    total_loss = 0
    for context, target in data:
        context_vector = make_context_vector(context, word_to_ix)  
        model.zero_grad()
        log_probs = model(context_vector)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))
        loss.backward()
        optimizer.step()

        total_loss += loss.data


Test the model

In [20]:
# ====================== TEST
context = ['People','create','to', 'direct']
context_vector = make_context_vector(context, word_to_ix)
a = model(context_vector).data.numpy()
print('Raw text: {}\n'.format(' '.join(raw_text)))
print('Context: {}\n'.format(context))
print('Prediction: {}'.format(get_max_prob_result(a[0], ix_to_word)))

Raw text: We are about to study the idea of a computational process. Computational processes are abstract beings that inhabit computers. As they evolve, processes manipulate other abstract things called data. The evolution of a process is directed by a pattern of rules called a program. People create programs to direct processes. In effect, we conjure the spirits of the computer with our spells.

Context: ['People', 'create', 'to', 'direct']

Prediction: programs


In [21]:
# ====================== TEST
context = ['to', 'direct','People','create']
context_vector = make_context_vector(context, word_to_ix)
a = model(context_vector).data.numpy()
print('Raw text: {}\n'.format(' '.join(raw_text)))
print('Context: {}\n'.format(context))
print('Prediction: {}'.format(get_max_prob_result(a[0], ix_to_word)))

Raw text: We are about to study the idea of a computational process. Computational processes are abstract beings that inhabit computers. As they evolve, processes manipulate other abstract things called data. The evolution of a process is directed by a pattern of rules called a program. People create programs to direct processes. In effect, we conjure the spirits of the computer with our spells.

Context: ['to', 'direct', 'People', 'create']

Prediction: programs


### Question: How do we interpret the output "a" of "model"?

Answer

The model's output is a LogSoftmax for all the vocabulary, so we are going to see a vector with 49 columns, each one representing the log probability of that word.


In [29]:
ix_to_word[a.argmax()]

'programs'

In [24]:
print(a)
print(a.shape)

[[-5.967013   -5.98879    -6.0427537  -7.261558   -5.8460364  -4.4023156
  -4.8269253  -5.5781717  -5.7604914  -5.0686755  -5.4758644  -5.0846686
  -4.9446383  -7.458612   -5.9991937  -5.5067034  -5.306475   -7.5814996
  -4.9813957  -7.169093   -5.9201093  -5.017847   -4.248554   -5.7780275
  -4.5697885  -6.1756883  -6.7498984  -6.539051   -5.6425614  -6.3343854
  -6.0594845  -3.7577832  -5.1462     -4.0101585  -5.7482953  -7.3520827
  -5.237054   -4.278888   -5.407286   -4.927964   -5.0052967  -2.9670312
  -3.9870594  -4.372441   -6.7310414  -4.7365365  -5.399485   -0.38324493
  -7.019016  ]]
(1, 49)


### Question: what is the word embedding of the word "People"?

In [37]:
model.get_word_embedding('People').shape

torch.Size([1, 100])

In [38]:
model.get_word_embedding('People')

tensor([[-0.0805,  0.6171, -1.5499,  2.4598,  0.7641,  1.0578,  0.6270,  0.1213,
         -0.8816,  0.0626,  0.6886, -1.1877, -0.2955, -0.3985,  0.4648,  0.7233,
         -0.8372, -0.1382,  0.5549,  1.1142, -0.2429,  1.9858,  1.1807,  0.4511,
          0.8436, -1.1676, -0.3438,  1.4283, -1.1930, -0.1008, -0.9379,  1.1222,
         -0.4656,  0.1642, -1.6222,  1.2438,  1.5218, -0.9276,  1.1824,  1.0896,
          1.1725,  0.3655, -0.5228, -0.1602, -0.7667,  0.0981,  0.3890,  0.0752,
         -2.0034, -0.0535, -1.0049, -0.8309, -0.4052, -0.3760,  0.6839, -1.4845,
         -1.3571, -0.8331, -1.1643,  0.7705,  0.7305,  1.8496, -1.0514,  0.7029,
          0.5374, -3.0386, -1.1142, -0.6691,  0.5549,  0.4782, -0.0376, -0.9259,
          1.2161, -1.0850, -0.0274,  0.3360, -2.0259, -0.5454,  0.8171,  0.0405,
         -1.1897, -0.4481, -0.7492,  0.6926, -0.8588, -2.3377, -1.6881, -0.1358,
          1.5038, -0.5620, -1.5067, -1.5304,  0.1643,  0.7635, -1.0298, -0.5326,
         -0.4690, -0.9940, -

### Question: in the lecture, the CBOW model can propose at least two kinds of embedding for a given word. Which one is used in this code?

**Answer**

We're using the input of the networks as embedding.

### Question: count all the parameters of the model. Is the embedding matrix learned during the training step?

In [41]:
# Count the parameters in this cell
pp=0
for p in list(model.parameters()):
    nn=1
    for s in list(p.size()):
      nn = nn*s
    pp += nn

print(pp)

24149
